Pneumonia is one of the leading respiratory illnesses worldwide, and its timely and accurate diagnosis is essential for effective treatment. Manually reviewing chest X-rays is a critical step in this process, and AI can provide valuable support by helping to expedite the assessment. In your role as a consultant data scientist, you will test the ability of a deep learning model to distinguish pneumonia cases from normal images of lungs in chest X-rays.

By fine-tuning a pre-trained convolutional neural network, specifically the ResNet-18 model, your task is to classify X-ray images into two categories: normal lungs and those affected by pneumonia. You can leverage its already trained weights and get an accurate classifier trained faster and with fewer resources.

## The Data

<img src="x-rays_sample.png" align="center"/>
&nbsp

You have a dataset of chest X-rays that have been preprocessed for use with a ResNet-18 model. You can see a sample of 5 images from each category above. Upon unzipping the `chestxrays.zip` file (code provided below), you will find your dataset inside the `data/chestxrays` folder divided into `test` and `train` folders. 

There are 150 training images and 50 testing images for each category, NORMAL and PNEUMONIA (300 and 100 in total). For your convenience, this data has already been loaded into a `train_loader` and a `test_loader` using the `DataLoader` class from the PyTorch library. 

In [9]:
# # Make sure to run this cell to use torchmetrics.
!pip install torch torchvision torchmetrics

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
# Import required libraries
# -------------------------
# Data loading
import random
import numpy as np
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Train model
import torch
from torchvision import models
import torch.nn as nn
import torch.optim as optim

# Evaluate model
from torchmetrics import Accuracy, F1Score

# Set random seeds for reproducibility
torch.manual_seed(69420)
np.random.seed(69420)
random.seed(69420)

In [11]:
import os
import zipfile

# Unzip the data folder
if not os.path.exists('data/chestxrays'):
    with zipfile.ZipFile('data/chestxrays.zip', 'r') as zip_ref:
        zip_ref.extractall('data')

In [12]:
# Define the transformations to apply to the images for use with ResNet-18
transform_mean = [0.485, 0.456, 0.406]
transform_std =[0.229, 0.224, 0.225]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize(mean=transform_mean, std=transform_std)])

# Apply the image transforms
train_dataset = ImageFolder('data/chestxrays/train', transform=transform)
test_dataset = ImageFolder('data/chestxrays/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=len(train_dataset) // 2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Initialize the model with pre-trained weights
weights = models.ResNet18_Weights.DEFAULT
model = models.resnet18(weights=weights)

# Modify the final layer to match the number of classes in the dataset
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'resnet18_chestxrays.pth')

Epoch 1/100, Loss: 0.5673511475324631
Epoch 2/100, Loss: 0.1481863111257553
Epoch 3/100, Loss: 0.07122110202908516
Epoch 4/100, Loss: 0.033084576949477196
Epoch 5/100, Loss: 0.012563369702547789
Epoch 6/100, Loss: 0.007501395419239998
Epoch 7/100, Loss: 0.0049491263926029205
Epoch 8/100, Loss: 0.003933176398277283
Epoch 9/100, Loss: 0.001745008456055075
Epoch 10/100, Loss: 0.0012855480890721083
Epoch 11/100, Loss: 0.0009064339101314545
Epoch 12/100, Loss: 0.0012319103989284486
Epoch 13/100, Loss: 0.0006734683702234179
Epoch 14/100, Loss: 0.0006155104347271845


### Below is the provided model evaluation code. Run the below cell to help you evaluate the accuracy and F1-score of your fine-tuned model.

In [8]:
#-------------------
# Evaluate the model
#-------------------

# Set model to evaluation mode
model.eval()

# Initialize metrics for accuracy and F1 score
accuracy_metric = Accuracy(task="binary")
f1_metric = F1Score(task="binary")

# Create lists to store all predictions and labels
all_preds = []
all_labels = []

# Disable gradient calculation for evaluation
with torch.no_grad():
    for inputs, labels in test_loader:
        # Forward pass
        outputs = model(inputs)
        preds = torch.sigmoid(outputs).round()  # Round to 0 or 1

        # Extend the lists with predictions and labels
        all_preds.extend(preds.squeeze().tolist())  # Squeeze to match the shape
        all_labels.extend(labels.tolist())  # No need to unsqueeze

# Convert lists back to tensors
all_preds = torch.tensor(all_preds).squeeze()  # Ensure the shape matches
all_labels = torch.tensor(all_labels).squeeze()  # Ensure the shape matches

# Ensure predictions and labels have the same shape
if all_preds.ndim > 1 and all_preds.shape[1] == 1:
    all_preds = all_preds.squeeze(1)

# Fix: Ensure predictions are in the same shape as labels
if all_preds.ndim > 1 and all_preds.shape[1] == 2:
    all_preds = all_preds[:, 1]  # Take the second column which represents the positive class

# Calculate accuracy and F1 score
test_acc = accuracy_metric(all_preds, all_labels).item()
test_f1 = f1_metric(all_preds, all_labels).item()

print(f"Test Accuracy: {test_acc}")
print(f"Test F1 Score: {test_f1}")

Test Accuracy: 0.550000011920929
Test F1 Score: 0.6896551847457886
